In [1]:
import csv
import numpy as np 
import random
import time
# sve_ivp
import os
import pandas as pd
from Update_LM_functions import calculate_constraints, update_lambda, openfile

In [2]:

## set path 
on_mac = False
on_thinkpad = False 
on_hpg = True

if on_mac == True: 
    data_path = "/Volumes/hodaakl/"
if on_thinkpad== True: 
    data_path = "//exasmb.rc.ufl.edu/blue/pdixit/hodaakl/"
if on_hpg == True: 
    data_path = "/blue/pdixit/hodaakl/"

# specify the project you are working on     
spec_folder_onServer = data_path + 'A5MCMC_IGF_FoxO/'
path = spec_folder_onServer + '1221_MuS_NewLM_newParRange/'

In [3]:
file_name_lambda = path + 'Lambdas.csv'
file_name_error = path+ 'Errors.csv'
read_dictionary = np.load(spec_folder_onServer + 'Arrays_for_max_ent/Cons_1213_Zero2ndMoments.npy',allow_pickle='TRUE').item()
real_cons = read_dictionary['array']


if not os.path.exists(file_name_lambda): 

    raise ValueError("No saved Lagrange multipliers, nothing to update. You have to same initial Lagrange multipliers first")
    
Lambda_np = openfile(file_name_lambda)

iterationp1, _ = Lambda_np.shape
iteration = iterationp1 -1

#
moments_filename =path + f'moments_{iteration}.csv'


df = pd.read_csv(moments_filename, sep = ',', header = None) 

Data_np = df.to_numpy()
### take away the nan values 
idxn = np.argwhere(np.isnan(Data_np))
idx_nan_rows = idxn[:,0]

data = np.delete(Data_np,idx_nan_rows, 0)
Preds = calculate_constraints(data)

# preds_power = np.floor(np.log10(Preds))
# alpha_power = np.ones(len(Preds))


In [6]:
print(Preds)
print(- np.log10(Preds))


[377.07500772 364.83213332 351.74104334 337.06462484 326.11928865
 322.5154479  319.11677987 377.07500772 360.9677675  345.24899438
 328.1721461  315.80656101 311.87421701 308.31458527 377.07500772
 357.7151279  339.89986475 320.97501092 307.63674316 303.53636642
 299.93643778 377.07500772 343.9856623  318.13542025 293.16425662
 277.70793626 273.50243875 270.08386035]
[-2.57642775 -2.56209308 -2.54622305 -2.52771318 -2.51337649 -2.50855052
 -2.50394964 -2.57642775 -2.55746842 -2.53813242 -2.51610172 -2.49942115
 -2.49397947 -2.48899407 -2.57642775 -2.55353731 -2.53135099 -2.50647122
 -2.4880382  -2.48221073 -2.47702923 -2.57642775 -2.53654034 -2.50261202
 -2.46711102 -2.44358829 -2.4369612  -2.43149863]


In [9]:
Error = Preds - real_cons[:len(Preds)]
Old_Lambda = Lambda_np[-1,:]
alpha_arr = 10**(-np.log10(Preds))
Lambda = update_lambda(Error = Error, old_lambda= Old_Lambda, alpha_arr = alpha_arr , true_constraints = real_cons, alpha_cons = 1)#, alpha_power = alpha_power) 
Lambda= Lambda.tolist()
Error = Error.tolist()



In [10]:
Lambda

[-0.0005220787481120768,
 -0.0005216366652179667,
 -0.0006310711203349235,
 -0.0006633038506191112,
 -0.0006193236679805134,
 -0.0005848341263278586,
 -0.0005971375796390529,
 -0.0005220787481120797,
 -0.0005973627015838935,
 -0.0006552901183991143,
 -0.0005854571624906612,
 -0.00048122701763059656,
 -0.0004068507453747698,
 -0.0004246208125216732,
 -0.000522078748112077,
 -0.0005096254816926763,
 -0.0006093333294628979,
 -0.000466714012531128,
 -0.00019536312482948314,
 -0.00012049951186918232,
 -0.0001462910464795287,
 -0.0005220787481120775,
 -0.000579513685866816,
 -0.0005595923686363614,
 0.00029028772836761163,
 0.0010791052016015281,
 0.0010713725010599343,
 0.0009125642147518546]

In [ ]:


# ------------------------------------------
#write the result 


if os.path.exists(file_name_error): 
    with open(file_name_error, 'a') as add_file_error:
        csv_adder_error = csv.writer(add_file_error, delimiter = ',')
        csv_adder_error.writerow(Error)
        add_file_error.flush()
else:
    with open(file_name_error, 'w') as new_file_error:

        csv_writer_error = csv.writer(new_file_error, delimiter = ',')
#         csv_writer_pars.writerow(Par_fieldnames)
        csv_writer_error.writerow(Error)
        new_file_error.flush()
    
#################################################### Storing the Lambda  

if os.path.exists(file_name_lambda): 
    with open(file_name_lambda, 'a') as add_file_lambda:
        csv_adder_lambda = csv.writer(add_file_lambda, delimiter = ',')
        csv_adder_lambda.writerow(Lambda)
        add_file_lambda.flush()
else:
    print('trouble loading file')


############################################# Plotting
import matplotlib.pyplot as plt
if os.path.isdir(path+'figs')==False:
    os.mkdir(path+'figs') 
# plot the absolute relative error
df_err = pd.read_csv(file_name_error, sep = ',', header = None) 
err_np = df_err.to_numpy()
rc_m= np.tile(real_cons[:len(err_np[0,:])] , [err_np.shape[0],1])
#print(rc_m.shape)
mean_err = np.mean(abs(err_np), axis = 1)
# mean_err.shape
real_abs = abs(err_np/rc_m)
mean_rel_abs = np.mean(real_abs, axis = 1)
# mean_err = np.mean(abs(err_np), axis = 1)
plt.plot(range(len(mean_rel_abs) ), mean_rel_abs)
plt.ylabel('Mean Abs relative Error ')
plt.xlabel('Iteration')
# plt.title('Iteration[1:]')
plt.savefig(path+'figs/error.png')
print('saved figure')
# save error array 
np.save(path+'mean_abs_error.npy', mean_rel_abs)
# plot the lagrange multipliers 

# plot the parameters 

# store the absolute relative error array 

